In [1]:
import pandas as pd
import numpy as np

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark import SparkContext
sc = SparkContext()

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
spark = SparkSession.builder.appName("Project-DNSC6290").getOrCreate()

In [5]:
import json # to convert json in df
from pandas.io.json import json_normalize # to normalize the json file
import os # it's a operational system library, to set some informations

In [6]:
#data = pd.read_csv("s3://project-data-large/train.csv")

/home/hadoop/miniconda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#data = pd.read_csv("train.csv")

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Read csv file from s3 and convert json to column

In [10]:
#pwd
#dir_path = "/Users/hyen86/BigData/Project/"
def load_df(df):
    # Columns that have json format
    json_columns = ['device', 'geoNetwork', 'totals', 'trafficSource']
    nrows = None
    
    df = pd.read_csv(df, 
                converters={column: json.loads for column in json_columns}, 
                dtype={'fullVisitorId': 'str'}, # Important!!
                nrows=nrows)
    
    for column in json_columns:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Shape: {df.shape}")
    return df

In [18]:
data = load_df("s3://project-data-large/train.csv")

/home/hadoop/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]


Shape: (903653, 55)


In [ ]:
from time import time
#check load_df running time
start_time = time()
data = load_df("s3://project-data-large/train.csv")
end_time = time()
time_taken = end_time - start_time
time_taken

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 55 columns):
 #   Column                                             Non-Null Count   Dtype 
---  ------                                             --------------   ----- 
 0   channelGrouping                                    903653 non-null  object
 1   date                                               903653 non-null  int64 
 2   fullVisitorId                                      903653 non-null  object
 3   sessionId                                          903653 non-null  object
 4   socialEngagementType                               903653 non-null  object
 5   visitId                                            903653 non-null  int64 
 6   visitNumber                                        903653 non-null  int64 
 7   visitStartTime                                     903653 non-null  int64 
 8   device.browser                                     903653 non-null  object
 9   devi

In [38]:
def missing_values(data):
    total = data.isnull().sum().sort_values(ascending = False) # getting the sum of null values and ordering
    percent = (data.isnull().sum() / data.isnull().count() * 100 ).sort_values(ascending = False) #getting the percent and order of null
    df = pd.concat([total, percent], axis=1, keys=['Total', 'Percent']) # Concatenating the total and percent
    print("Total columns at least one Values: ")
    print(df.loc[df['Total'] >0])
    return 

In [39]:
# From above function, we can see some columns are having a lot of null values. 
# Therefore, we deceided to drop all columns that have more than 50% null vlaue.
# We drop all the trafficSource related columns.
missing_values(data)

Total columns at least one Values: 
                                               Total    Percent
trafficSource.campaignCode                    903652  99.999889
trafficSource.adContent                       892707  98.788694
totals.transactionRevenue                     892138  98.725728
trafficSource.adwordsClickInfo.isVideoAd      882193  97.625195
trafficSource.adwordsClickInfo.adNetworkType  882193  97.625195
trafficSource.adwordsClickInfo.slot           882193  97.625195
trafficSource.adwordsClickInfo.page           882193  97.625195
trafficSource.adwordsClickInfo.gclId          882092  97.614018
trafficSource.isTrueDirect                    629648  69.678073
trafficSource.referralPath                    572712  63.377425
trafficSource.keyword                         502929  55.655102
totals.bounces                                453023  50.132407
totals.newVisits                              200593  22.198012
totals.pageviews                                 100   0.011066


In [45]:
dataNew = data.drop(data.columns[[38, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]], axis=1)

In [46]:
dataNew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 40 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   channelGrouping                   903653 non-null  object
 1   date                              903653 non-null  int64 
 2   fullVisitorId                     903653 non-null  object
 3   sessionId                         903653 non-null  object
 4   socialEngagementType              903653 non-null  object
 5   visitId                           903653 non-null  int64 
 6   visitNumber                       903653 non-null  int64 
 7   visitStartTime                    903653 non-null  int64 
 8   device.browser                    903653 non-null  object
 9   device.browserVersion             903653 non-null  object
 10  device.browserSize                903653 non-null  object
 11  device.operatingSystem            903653 non-null  object
 12  de

## Convert Json Columns

In [31]:
data2 = spark.read.option("header",True).csv("s3://project-data-large/train.csv")
data2.printSchema()
data2

root
 |-- channelGrouping: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device: string (nullable = true)
 |-- fullVisitorId: string (nullable = true)
 |-- geoNetwork: string (nullable = true)
 |-- sessionId: string (nullable = true)
 |-- socialEngagementType: string (nullable = true)
 |-- totals: string (nullable = true)
 |-- trafficSource: string (nullable = true)
 |-- visitId: string (nullable = true)
 |-- visitNumber: string (nullable = true)
 |-- visitStartTime: string (nullable = true)



DataFrame[channelGrouping: string, date: string, device: string, fullVisitorId: string, geoNetwork: string, sessionId: string, socialEngagementType: string, totals: string, trafficSource: string, visitId: string, visitNumber: string, visitStartTime: string]

In [33]:
type(data2)

pyspark.sql.dataframe.DataFrame

In [34]:
data2.show(10)

+---------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|channelGrouping|    date|              device|       fullVisitorId|          geoNetwork|           sessionId|socialEngagementType|              totals|       trafficSource|             visitId|         visitNumber|      visitStartTime|
+---------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| Organic Search|20160902|"{""browser"": ""...| ""browserVersion...| ""browserSize"":...| ""operatingSyste...| ""operatingSyste...| ""isMobile"": false| ""mobileDeviceBr...| ""mobileDeviceMo...| ""mobileInputSel...| ""mobileDeviceIn...|
| Organic Search|20160902|"{""browser"": ""...| ""br

In [35]:
data2.createOrReplaceTempView("data2")

In [36]:
json_to_csv =  data2.select(col("channelGrouping"),from_json(col("device"),schema).as("device2")).select("channelGrouping","device2.*")
json_to_csv.printSchema()
json_to_csv.show()

SyntaxError: invalid syntax (<ipython-input-36-c9324dafd63c>, line 1)

In [25]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [26]:
from pyspark.sql import Column
pure_json_df = sqlContext.createDataFrame([Column(json=str(data["device"])),
                                             Column(json=str(data["geoNetwork"])),
                                             Column(json=str(data["totals"])),
                                             Column(json=str(data["trafficSource"]))])

KeyError: 'device'

In [86]:
#Convert json to dataset
new_df = sqlContext.read.json(pure_json_df.rdd.map(lambda r: r.json))

In [87]:
new_df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)



In [88]:
new_df.show()

+--------------------+
|     _corrupt_record|
+--------------------+
|0         {"brows...|
|0         {"conti...|
|0         {"visit...|
|0         {"campa...|
+--------------------+



In [45]:
a = sqlContext.read.json(data)

TypeError: path can be only string, list or RDD

In [44]:
sqlContext.read.json(data.select("device").rdd.map(_.getAs[String](0)))

AttributeError: 'DataFrame' object has no attribute 'select'

In [18]:
import multiprocessing as mp
pool = mp.Pool(3)
data_json = pool.apply(load_df(data))

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U30'), dtype('<U30')) -> dtype('<U30')

## Creating the function to handle with date

In [9]:
# library of datetime
from datetime import datetime

# This function is to extract date features
def date_process(df):
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d") # seting the column as pandas datetime
    df["_weekday"] = df['date'].dt.weekday #extracting week day
    df["_day"] = df['date'].dt.day # extracting day
    df["_month"] = df['date'].dt.month # extracting day
    df["_year"] = df['date'].dt.year # extracting day
    df['_visitHour'] = (df['visitStartTime'].apply(lambda x: str(datetime.fromtimestamp(x).hour))).astype(int)
    
    return df #returning the df after the transformations

In [ ]:
data2 = date_process(data)

In [ ]:
data2.head(10)

In [51]:
#DataFrame
df_data = spark.read.format('csv').option('header','true').option('inferSchema','true').load("train.csv")

In [6]:
df_data.head(10) #It's a DataFrame

[Row(channelGrouping='Organic Search', date=20160902, device='"{""browser"": ""Chrome""', fullVisitorId=' ""browserVersion"": ""not available in demo dataset""', geoNetwork=' ""browserSize"": ""not available in demo dataset""', sessionId=' ""operatingSystem"": ""Windows""', socialEngagementType=' ""operatingSystemVersion"": ""not available in demo dataset""', totals=' ""isMobile"": false', trafficSource=' ""mobileDeviceBranding"": ""not available in demo dataset""', visitId=' ""mobileDeviceModel"": ""not available in demo dataset""', visitNumber=' ""mobileInputSelector"": ""not available in demo dataset""', visitStartTime=' ""mobileDeviceInfo"": ""not available in demo dataset""'),
 Row(channelGrouping='Organic Search', date=20160902, device='"{""browser"": ""Firefox""', fullVisitorId=' ""browserVersion"": ""not available in demo dataset""', geoNetwork=' ""browserSize"": ""not available in demo dataset""', sessionId=' ""operatingSystem"": ""Macintosh""', socialEngagementType=' ""operat

In [42]:
df_data_jason = df_data.withColumn(lambda x: (load_df(x,nrows=None)))

TypeError: withColumn() missing 1 required positional argument: 'col'

In [52]:
df2 = df_data.rdd.flatMap(lambda x: (load_df(x,nrows=None))).toDF()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 24, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/rdd.py", line 1354, in takeUpToNumLeft
    yield next(iterator)
  File "

In [53]:
#RDD
df_data_RDD = sc.textFile("train.csv")

In [54]:
df_data_RDD

train.csv MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:0

In [92]:
df_data_RDD_jason = df_data_RDD.map(lambda x: (load_df(x,nrows=None)))

In [93]:
df_data_RDD_jason.take(10)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 20.0 failed 1 times, most recent failure: Lost task 0.0 in stage 20.0 (TID 71, huang-chins-mbp.fios-router.home, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 271, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/rdd.py", line 1440, in takeUpToNumLeft
    yield next(iterator)
  File "/usr/local/opt/apache-spark/libexec/pyt

In [46]:
sc.stop()

In [47]:
spark.stop()